&copy; 2024 by Pearson Education, Inc. All Rights Reserved. The content in this notebook is based on the book [**Python for Programmers**](https://amzn.to/2VvdnxE).

In [ ]:
# enable high-res images in notebook 
%config InlineBackend.figure_format = 'retina'

In [ ]:
%%html
<!-- CSS settings for this notbook -->
<style>
    h1 {color:#BB0000}
    h2 {color:purple}
    h3 {color:#0099ff}
    hr {    
        border: 0;
        height: 3px;
        background: #333;
        background-image: linear-gradient(to right, #ccc, black, #ccc);
    }
</style>

# 16. Big Data: Hadoop, Spark, NoSQL and IoT 


# 16.8 Internet of Things and Dashboards

| Small subset of IoT device types and applications |
| --- |
| **activity trackers**—Apple Watch, FitBit, …
| **personal assistants**—Amazon Echo (Alexa), Apple HomePod (Siri), Google Home (Google Assistant)
| **appliances**—ovens, coffee makers, refrigerators, …
| **driverless cars**
| **earthquake sensors**
| **healthcare**—blood glucose monitors for diabetics, blood pressure monitors, electrocardiograms (EKG/ECG), electroencephalograms (EEG), heart monitors, ingestible sensors, pacemakers, sleep trackers, …
| **sensors**—chemical, gas, GPS, humidity, light, motion, pressure, temperature, …
| **smart home**—lights, garage openers, video cameras, doorbells, irrigation controllers, security devices, smart locks, smart plugs, smoke detectors, thermostats, air vents
| **tracking devices**
| **wireless network devices**

<hr style="height:2px; border:none; color:#000; background-color:#000;">

## Publish and Subscribe 
* **IoT devices** (and more) commonly communicate via **pub/sub (publisher/subscriber) systems**
* **Publisher** &mdash; Anything that **sends a message** to a **cloud-based service**, which in turn **sends** that **message** to all **subscribers**
    * **Publisher** specifies a **topic** or **channel**
* **Subscriber** specifies one or more **topics** or **channels** for which they’d like to **receive messages**

<hr style="height:2px; border:none; color:#000; background-color:#000;">

## Visualizing a PubNub Sample Live Stream 
* **PubNub** is geared to **real-time pub/sub applications**
    * Up to 1 million free transactions per month
    * https://www.pubnub.com/pricing/
* Many **use-cases** 
    * IoT, chat, online multiplayer games, social apps, collaborative apps
* Provides several **demo live streams**, including one that **simulates IoT sensors** 
* Common to **visualize** live data **streams** for **monitoring purposes**

<hr style="height:2px; border:none; color:#000; background-color:#000;">

## Creating a Python PubNub Subscriber (1 of 2)
* **`pubnub` module** for performing **pub/sub operations**
* Provides several [**sample streams for you to experiment with**](https://www.pubnub.com/demos/real-time-data-streaming/?show=demo), including
	* **Twitter** live feed
	* **Wikipedia Changes**
	* **Game State Sync**—**Simulated**: multiplayer game data
	* **Sensor Network**—**Simulated sensor data**: radiation, humidity, temperature and ambient light
	* **Market Orders**: **Simulated stock orders** for five fake companies

<hr style="height:2px; border:none; color:#000; background-color:#000;">

## Creating a Python PubNub Subscriber (2 of 2)
* We'll **subscribe** to their **Market Orders stream**, then **visualize** changing stock prices in a **barplot**
* `pip install "pubnub>=7.3.2"`
* [PubNub Python SDK documentation](https://www.pubnub.com/docs/sdks/python)
* **`stocklistener.py`** subscribes to the stream and visualizes the stock prices

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Message Format
* **Simulated Market Orders** stream returns **JSON objects** containing five key–value pairs with the keys **`'bid_price'`**, **`'order_quantity'`**, **`'symbol'`**, **`'timestamp'`** and **`'trade_type'`**
    * We’ll use `'bid_price'` and `'symbol'`
* **PubNub client** returns **JSON objects** as Python **dictionaries**

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Importing the Libraries
```python
# stocklistener.py
"""Visualizing a PubNub live stream."""
from matplotlib import animation
import matplotlib.pyplot as plt
import pandas as pd
import random 
import seaborn as sns
import sys
import keys

from pubnub.callbacks import SubscribeCallback
from pubnub.enums import PNStatusCategory
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub

```

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Class `StockSubscriberCallback`
* A PubNub stream **listener** receives **status notifications** and **messages from the channel**
* Subclass of **`SubscribeCallback`** (module `pubnub.callbacks`)
* **Overridden method `status`** &mdash; called by PubNub client each time a **status notification arrives**
    * We check for **subscribed to** or **unsubscribed from** a channel messages
* **Overridden method `message`** &mdash; called when a **message arrives from the channel**
    * Stores new stock price

```python
class StockSubscriberCallback(SubscribeCallback):
    """StockSubscriberCallback receives messages from PubNub."""
    def __init__(self, df, limit=1000):
        """Create instance variables for tracking number of tweets."""
        self.df = df  # DataFrame to store last stock prices
        self.order_count = 0
        self.MAX_ORDERS = limit  # 1000 by default
        super().__init__()  # call superclass's init
```

```python
    def status(self, pubnub, status):
        if status.category == PNStatusCategory.PNConnectedCategory:
            print('Connected to PubNub')
        elif status.category == PNStatusCategory.PNAcknowledgmentCategory:
            print('Disconnected from PubNub')
```

```python
    def message(self, pubnub, message):
        symbol = message.message['symbol']
        bid_price = message.message['bid_price']
        print(symbol, bid_price)
        self.df.at[companies.index(symbol), 'price'] = bid_price
        self.order_count += 1
        
        # if MAX_ORDERS is reached, unsubscribe from PubNub channel
        if self.order_count == self.MAX_ORDERS:
            pubnub.unsubscribe_all()
```

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Function `update` Visualizes the Stock Prices 

```python
def update(frame_number):
    """Configures bar plot contents for each animation frame."""
    plt.cla()  # clear old barplot
    axes = sns.barplot(
        data=companies_df, x='company', y='price', palette='cool')
    axes.set(xlabel='Company', ylabel='Price')  
    plt.tight_layout()
```

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Configuring the Application

```python
if __name__ == '__main__':
    sns.set_style('whitegrid')  # white background with gray grid lines
    figure = plt.figure('Stock Prices')  # Figure for animation
```

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Configuring the PubNub Client
* Specify the **PubNub subscription key**
    * **Used with the channel name** to **subscribe to the channel**
* The `SensorSubscriberCallback` object is passed to the **`PubNub` client’s `add_listener` method** to register it to **receive messages from the channel**

```python
    # set up pubnub-market-orders sensor stream key
    config = PNConfiguration()
    config.subscribe_key = 'sub-c-4377ab04-f100-11e3-bffd-02ee2ddab7fe'
    config.user_id = keys.pubnub_user_id 

    # create PubNub client and register a SubscribeCallback
    pubnub = PubNub(config) 
    pubnub.add_listener(
        StockSubscriberCallback(df=companies_df, 
            limit=int(sys.argv[1] if len(sys.argv) > 1 else 1000))
```

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Subscribing to the Channel
* **Completes subscription process** by indicating that we wish to **receive messages** from **channel `'pubnub-market-orders'`**
* **`execute()`** tells client to **begin listening** for messages

```python
    # subscribe to pubnub-sensor-network channel and begin streaming
    pubnub.subscribe().channels('pubnub-market-orders').execute()
```

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Configuring the FuncAnimation and Displaying the Window
* **Matplotlib’s `show` method** normally **blocks** a script from continuing until you close the `Figure`
    * **`block=False`** allows execution to continue 
    * we'll **configure Pubnub client** next
* For **detailed intro to Matplotlib `FuncAnimation`**, see my [**Python Fundamentals LiveLessons videos**](https://learning.oreilly.com/videos/python-fundamentals/9780135917411/9780135917411-PFLL_Lesson06_17) (two videos) and in [**Python for Programmers, Section 6.4**](https://learning.oreilly.com/library/view/python-for-programmers/9780135231364/ch06.xhtml#ch06lev1sec4)

```python
    # configure and start animation that calls function update
    stock_animation = animation.FuncAnimation(
        figure, update, repeat=False, interval=33)
    plt.show()  # keeps graph on screen until you dismiss its window
```

In [ ]:
%matplotlib widget

In [ ]:
# stocklistener.py
"""Visualizing a PubNub live stream."""
from matplotlib import animation
import matplotlib.pyplot as plt
import pandas as pd
import random 
import seaborn as sns
import sys
import uuid

from pubnub.callbacks import SubscribeCallback
from pubnub.enums import PNStatusCategory
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub

companies = ['Apple', 'Bespin Gas', 'Elerium', 'Google', 'Linen Cloth']

# DataFrame to store last stock prices 
companies_df = pd.DataFrame(
    {'company': companies, 'price' : [0, 0, 0, 0, 0]})
 
class SensorSubscriberCallback(SubscribeCallback):
    """SensorSubscriberCallback receives messages from PubNub."""
    def __init__(self, df, limit=1000):
        """Create instance variables for tracking number of tweets."""
        self.df = df  # DataFrame to store last stock prices
        self.order_count = 0
        self.MAX_ORDERS = limit  # 1000 by default
        super().__init__()  # call superclass's init

    def status(self, pubnub, status):
        if status.category == PNStatusCategory.PNConnectedCategory:
            print('Subscribed')
        elif status.category == PNStatusCategory.PNAcknowledgmentCategory:
            print('Unsubscribed')
 
    def message(self, pubnub, message):
        symbol = message.message['symbol']
        bid_price = message.message['bid_price']
        print(symbol, bid_price)
        self.df.at[companies.index(symbol), 'price'] = bid_price
        self.order_count += 1
        
        # if MAX_ORDERS is reached, unsubscribe from PubNub channel
        if self.order_count == self.MAX_ORDERS:
            pubnub.unsubscribe_all()
            
def update(frame_number):
    """Configures bar plot contents for each animation frame."""
    plt.cla()  # clear old barplot
    axes = sns.barplot(
        data=companies_df, x='company', y='price', palette='cool') 
    axes.set(xlabel='Company', ylabel='Price')  

#if __name__ == '__main__':
sns.set_style('whitegrid')  # white background with gray grid lines
figure = plt.figure('Stock Prices')  # Figure for animation

# set up pubnub-market-orders sensor stream key
config = PNConfiguration()
config.subscribe_key = 'sub-c-99084bc5-1844-4e1c-82ca-a01b18166ca8'
config.uuid = 'UUID_DeitelHeartbeatUnitTest' # new requirement in SDK 6.x

# create PubNub client and register a SubscribeCallback
pubnub = PubNub(config) 
pubnub.add_listener(
    SensorSubscriberCallback(df=companies_df, 
        limit=1000)) #int(sys.argv[1] if len(sys.argv) > 1 else 1000)))

# subscribe to pubnub-sensor-network channel and begin streaming
pubnub.subscribe().channels('pubnub-market-orders').execute()

# configure and start animation that calls function update
stock_animation = animation.FuncAnimation(
    figure, update, frames=1000, repeat=False, interval=33)
plt.tight_layout()
plt.show()  # keeps graph on screen until you dismiss its window


#**************************************************************************
#* (C) Copyright 1992-2018 by Deitel & Associates, Inc. and               *
#* Pearson Education, Inc. All Rights Reserved.                           *
#*                                                                        *
#* DISCLAIMER: The authors and publisher of this book have used their     *
#* best efforts in preparing the book. These efforts include the          *
#* development, research, and testing of the theories and programs        *
#* to determine their effectiveness. The authors and publisher make       *
#* no warranty of any kind, expressed or implied, with regard to these    *
#* programs or to the documentation contained in these books. The authors *
#* and publisher shall not be liable in any event for incidental or       *
#* consequential damages in connection with, or arising out of, the       *
#* furnishing, performance, or use of these programs.                     *
#**************************************************************************    
    
    



<hr style="height:2px; border:none; color:#000; background-color:#000;">

## Simulating an Internet-Connected Thermostat in Python (1 of 2)
* Common to use **IoT simulators** for **testing**, especially if you **do not have access to actual devices and sensors** during development
* Many **cloud vendors** have **IoT simulation** capabilities
    * **IBM Watson IoT Platform**, **IOTIFY.io**, ...
* We'll create a **script** that **simulates IoT thermostat** 
    * Uses **PubNub** free tier to publish **JSON messages**

<hr style="height:2px; border:none; color:#000; background-color:#000;">

## Simulating an Internet-Connected Thermostat in Python (2 of 2)
* We'll simulate a **temperature sensor** that can issue
    * **low-temperature warnings** before pipes freeze
    * **high-temperature warnings** to indicate there might be a fire
* Our messages will contain 
    * **location** 
    * **temperature**
    * **low** or **high temperature warnings** if the temperature drops to **3 degrees Celsius** or rises to **35 degrees Celsius**
* Use Plotly's **`dash`** to create a simple **dashboard**

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Installing Libraries
* `pip install dash plotly`
* [Plotly's Dash documentation](https://dash.plotly.com/)

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Invoking the `simulator_pubnub.py` Script
* Script `simulator_pubnub.py` simulates our thermostat 
* Invoke script with two command-line arguments

> `ipython simulator_pubnub.py 1000 1`

* **number of total messages** to simulate 
* **delay** in seconds **between sending dweets**
* Can **immediately begin tracking** messages on `pubnub.com` site at 
> https://admin.pubnub.com/
* Go to **Debug Console**
* Set active channel to **`deitel-thermostat-simulator`** and click **UPDATE**
* Delete the default client panels
* Click **+ Create Client** — messages should begin appearing in the new client

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Publishing Messages 

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Script That Publishes Messages Through PubNub 
 
```python
# simulator_pubnub.py
"""A connected thermostat simulator that publishes JSON
messages using PubNub"""
import keys
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub
from pubnub.exceptions import PubNubException
import random
import sys
import time

MIN_CELSIUS_TEMP = -25  
MAX_CELSIUS_TEMP = 45 
MAX_TEMP_CHANGE = 2

# get the number of messages to simulate and delay between them
NUMBER_OF_MESSAGES = int(sys.argv[1]) 
MESSAGE_DELAY = int(sys.argv[2])

# PubNub configuration
config = PNConfiguration()
config.user_id = keys.pubnub_user_id
config.publish_key = keys.pubnub_publish_key 
config.subscribe_key = keys.pubnub_subscribe_key 
pubnub = PubNub(config)

channel = 'deitel-thermostat-simulator'  # provide a unique name

thermostat = {'Location': 'Home',
              'Temperature': 20, 
              'Too_Low': False,
              'Too_High': False}

print('Temperature simulator starting')

def publish_callback(envelope, status):
    # Handle PUBLISH response and status
    if status.is_error():
        print(f'Failed to publish message: {status}')

for message in range(1, NUMBER_OF_MESSAGES + 1):
    # generate a random number in the range -MAX_TEMP_CHANGE 
    # through MAX_TEMP_CHANGE and add it to the current temperature
    thermostat['Temperature'] += random.randrange(
        -MAX_TEMP_CHANGE, MAX_TEMP_CHANGE + 1)
    
    # ensure that the temperature stays within range
    if thermostat['Temperature'] < MIN_CELSIUS_TEMP:
        thermostat['Temperature'] = MIN_CELSIUS_TEMP
    
    if thermostat['Temperature'] > MAX_CELSIUS_TEMP:
        thermostat['Temperature'] = MAX_CELSIUS_TEMP
    
    # check for low temperature warning
    if thermostat['Temperature'] < 3:
        thermostat['Too_Low'] = True
    else:
        thermostat['Too_Low'] = False

    # check for high temperature warning
    if thermostat['Temperature'] > 35:
        thermostat['Too_High'] = True
    else:
        thermostat['Too_High'] = False

    # Publish the message to PubNub
    try:
        pubnub.publish().channel(channel).message(
            thermostat).pn_async(publish_callback)
    except PubNubException as e:
        print(f"An error occurred: {e}")
        sys.exit(1)
    else:
        print(f'Message {message}: {thermostat}\r', end='')
        
    time.sleep(MESSAGE_DELAY)

print('\nTemperature simulator finished')
```

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Script That Subscribes to the Thermostat Messages and Visualizes them Using Plotly's Dash 
* `ipython simpulator_dashboard.py`
* View the dashboard at http://localhost:8050
 
```python
# simulator_dashboard.py
"""Script That Subscribes to the Thermostat Messages 
and Visualizes them Using Plotly's Dash"""
import dash
from dash import dcc, html, Output, Input
import plotly.graph_objs as go
from pubnub.enums import PNStatusCategory
from pubnub.pubnub import PubNub
from pubnub.pnconfiguration import PNConfiguration
from pubnub.callbacks import SubscribeCallback
import keys 

# create Dash app named with the global variable __name__ 
app = dash.Dash(__name__)

# create app's layout
app.layout = html.Div([
    dcc.Graph(id='temperature-gauge', style={'marginBottom': '0px'}),  
    html.Div(id='fahrenheit-text', 
             style={'textAlign': 'center', 
                    'fontSize': 30, 'fontFamily': 'Arial'}),
    html.Div([
        html.Div([
            html.Div(id='low-warning-light', 
                     style={'display': 'inline-block', 'width': '20px', 
                            'height': '20px', 'borderRadius': '50%'}),
            html.Div("Too Low", 
                     style={'display': 'inline-block', 
                            'fontSize': 16, 'fontFamily': 'Arial'})
        ], style={'textAlign': 'center', 'marginTop': '10px'}),
        html.Div([
            html.Div(id='high-warning-light', 
                     style={'display': 'inline-block', 'width': '20px', 
                            'height': '20px', 'borderRadius': '50%'}),
            html.Div("Too High", 
                     style={'display': 'inline-block', 
                            'fontSize': 16, 'fontFamily': 'Arial'})
        ], style={'textAlign': 'center', 
                  'marginTop': '10px'})
    ]),
    dcc.Interval(id='interval-component', 
                 interval=1000)  # refresh once per second
], style={'fontFamily': 'Arial', 'width': '500px', 'margin': '0 auto'})

# global variable to store latest message received from thermostat
latest_message = None

class ThermostatSubscribeCallback(SubscribeCallback):
    """ThermostatSubscribeCallback receives messages from PubNub."""
    
    def __init__(self, max_messages=1000):
        """Creates instance variables for tracking number of messages."""
        self.message_count = 0
        self.MAX_MESSAGES = max_messages  # 1000 by default
        super().__init__()  # call superclass's init

    def status(self, pubnub, status):
        """Processes status notifications from PubNub."""
        if status.category == PNStatusCategory.PNConnectedCategory:
            print('Subscribed')
        elif status.category == PNStatusCategory.PNAcknowledgmentCategory:
            print('Unsubscribed')
        else:
            print(status.category)
 
    def message(self, pubnub, message):
        """Receives each message PubNub pushes."""
        global latest_message
        self.message_count += 1
        latest_message = message.message
         
        # if MAX_MESSAGES reached, unsubscribe from PubNub channel
        if self.message_count == self.MAX_MESSAGES:
            pubnub.unsubscribe_all()
         
# set up the Dash app's callback that updates the UI 
@app.callback(
    [Output('temperature-gauge', 'figure'),
     Output('fahrenheit-text', 'children'),
     Output('low-warning-light', 'style'),
     Output('high-warning-light', 'style')],
    [Input('interval-component', 'n_intervals')]
)
def update_dashboard(n):
    # do not update unless latest_message contains a message
    if latest_message is None:
        raise dash.exceptions.PreventUpdate

    # get the temperature from the message and convert it to Fahrenheit
    temperature_c = latest_message.get('Temperature', 0)
    temperature_f = temperature_c * 9 / 5 + 32
    fahrenheit_text = f"{temperature_f:.1f}°F"

    # create the gauge graph object (go) for the next update
    gauge = go.Figure(go.Indicator(
        mode="gauge+number",
        value=temperature_c,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Temperature (°C)"},
        number={'font': {'family': "Arial"}}, 
        gauge={
            'axis': {'range': [-25, 45], 
                     'tickfont': {'size': 12, 'family': 'Arial'}},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [-25, 0], 'color': 'cyan'},
                {'range': [0, 45], 'color': 'orange'}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': temperature_c
            }
        }
    ))

    # update warning lights based on values in latest message
    low_light = latest_message.get('Too_Low', False)
    high_light = latest_message.get('Too_High', False)

    low_light_style = {
        'display': 'inline-block',
        'width': '20px', 
        'height': '20px', 
        'borderRadius': '50%', 
        'backgroundColor': 'red' if low_light else 'grey',
        'marginRight': '10px'
    }
    high_light_style = {
        'display': 'inline-block',
        'width': '20px', 
        'height': '20px', 
        'borderRadius': '50%', 
        'backgroundColor': 'red' if high_light else 'grey',
        'marginRight': '10px'
    }

    return gauge, fahrenheit_text, low_light_style, high_light_style

# launch Dash server when this file is run as a script
if __name__ == '__main__':
    # PubNub client subscription info
    config = PNConfiguration()
    config.subscribe_key = keys.pubnub_subscribe_key
    config.user_id = keys.pubnub_user_id # your login ID
    
    # create the pubnub client
    pubnub = PubNub(config)
    
    # channel we'll subscribe to
    CHANNEL_NAME = 'deitel-thermostat-simulator'
    
    # set up the listener and subscribe to the channel
    pubnub.add_listener(ThermostatSubscribeCallback())
    pubnub.subscribe().channels(CHANNEL_NAME).execute()

    app.run_server(debug=True)
```

### `app.layout` defines structure and appearance of the dashboard
* `html.Div([...], style={'fontFamily': 'Arial', 'width': '500px', 'margin': '0 auto'})`
    * Main container for app's layout
    * Subcontainers define various dashboard components
* `dcc.Graph(id='temperature-gauge', style={'marginBottom': '0px'})`
    * Creates a graph to display the temperature gauge
    * `id` `'temperature-gauge'` links used by callback function that updates the gauge
* `dcc.Interval(id='interval-component', interval=1000)`
    * triggers dashboard update every 1000 milliseconds
* `id` properties of components like `'temperature-gauge'`, `'fahrenheit-text'`, `'low-warning-light'`, `'high-warning-light'` are referenced in the `@app.callback` decorator to dynamically update these components based on data received from the PubNub subscription

### `@app.callback` decorator and `update_dashboard` function dynamically update the dashboard
* `@app.callback` Decorator
    * Defines a callback function that updates the dashboard in response to input changes
    * Connects callback function's inputs and outputs to specific app components 
    * Outputs: `Output('id', 'property')` specifies which component property to update
        * `'temperature-gauge', 'figure'` updates the `figure` of the `temperature-gauge`.
        * `'fahrenheit-text', 'children'` updates text (`children`) of `div` `fahrenheit-text`
        * `'low-warning-light', 'style'` updates style of `div` `low-warning-light`
        * `'high-warning-light', 'style'` updates style `div` `high-warning-light`
    * Inputs: `Input('component_id', 'component_property')` determines what triggers the callback
        * `'interval-component', 'n_intervals'` triggers callback every time the interval defined by `dcc.Interval` completes
* `update_dashboard` Function
    * Parameter is number of intervals that have passed
    * Not used but necessary to receive callback's input is `n_intervals` property of `Interval` component
    * Creates temperature gauge using Plotly's `go.Indicator`
    * Updates styles of warning lights based on `Too_Low` or `Too_High`
    * Returns to Dash a tuple of outputs specified in the `@app.callback`
        * Figure for temperature gauge
        * Text for Fahrenheit temperature
        * Styles for low and high warning lights
    * Dash updates the onscreen presentation

### `go.Indicator`
* From Plotly's Graph Objects (`go`) library
* For gauges/dials that display single values
* `mode="gauge+number"`: indicator displays gauge + numeric value
* `value=temperature_c`: current value to display
* `domain={'x': [0, 1], 'y': [0, 1]}`: use plot's full available area 
* `title={'text': "Temperature (°C)"}`: gauge title 
* `number={'font': {'family': "Arial"}}`: customizes number's appearance
* Gauge customizations
    * `axis`: defines range of values and tick label attributes
    * `bar`: defines appearance of bar indicating current value
    * `steps`: subranges on the gauge, each with its own color
    * `threshold`: draws a line with specified attributes at a specific value on the gauge

<hr style="height:2px; border:none; color:#000; background-color:#000;">

# Resources
### Many Free Big-Data Sources
* Articles and sites with links to **hundreds of free big data sources**

| Big-data sources |
| :--- |
| [**“The Best Tools for Using Twitter as a Data Source”**](https://www.methodspace.com/best-tools-twitter-data-source) |
| [**“How to Use Wikipedia as a Data Source”**](https://www.thedataschool.co.uk/jeremy-kneebone/use-wikipedia-data-source/) |
| [**“Awesome-Public-Datasets”**](https://github.com/caesar0301/awesome-public-datasets) |
| [**“AWS Public Datasets”**](https://aws.amazon.com/public-datasets/) |
| [**“Big Data And AI: 30 Amazing (And Free) Public Data Sources For 2018,”** by B. Marr](https://www.forbes.com/sites/bernardmarr/2018/02/26/big-data-and-ai-30-amazing-and-free-public-data-sources-for-2018/) |
| [**“Datasets for Data Mining and Data Science”**](http://www.kdnuggets.com/datasets/index.html) |
| [**“Exploring Open Data Sets”**](https://datascience.berkeley.edu/open-data-sets/) |
| [**“Free Big Data Sources”**](http://datamics.com/free-big-data-sources/) |
| [**_Hadoop Illuminated_, Chapter 16. Publicly Available Big Data Sets**](http://hadoopilluminated.com/hadoop_illuminated/Public_Bigdata_Sets.html) |
| [**“List of Public Data Sources Fit for Machine Learning”**](https://blog.bigml.com/list-of-public-data-sources-fit-for-machine-learning/) |
| [**“Open Data,”** Wikipedia](https://en.wikipedia.org/wiki/Open_data) |
| [**“Open Data 500 Companies**”](http://www.opendata500.com/us/list/) |
| [**“Other Interesting Resources/Big Data and Analytics Educational Resources and Research,”**, Bernard Marr](http://computing.derby.ac.uk/bigdatares/?page_id=223) |
| [**“6 Amazing Sources of Practice Data Sets”**](https://www.jigsawacademy.com/6-amazing-sources-of-practice-data-sets/) |
| [**“20 Big Data Repositories You Should Check Out”** M. Krivanek](http://www.datasciencecentral.com/profiles/blogs/20-free-big-data-sources-everyone-should-check-out) |
| [**“70+ Websites to Get Large Data Repositories for Free”**](http://bigdata-madesimple.com/70-websites-to-get-large-data-repositories-for-free/) |
| [**“Ten Sources of Free Big Data on Internet,”** A. Brown](https://www.linkedin.com/pulse/ten-sources-free-big-data-internet-alan-brown) |
| [**“Top 20 Open Data Sources”**](https://www.linkedin.com/pulse/top-20-open-data-sources-zygimantas-jacikevicius) |
| [**“We’re Setting Data, Code and APIs Free,”** NASA](https://open.nasa.gov/open-data/) |
| [**“Where Can I Find Large Datasets Open to the Public?”** Quora](https://www.quora.com/Where-can-I-find-large-datasets-open-to-the-public) |

<hr style="height:2px; border:none; color:#000; background-color:#000;">

### Kaggle Competition Site 
* **No obvious optimal solutions** for many **machine learning** and **deep learning tasks**
* People’s **creativity is really the only limit**
* Companies and organizations **fund competitions** 
    * Encourage people worldwide to **develop better-performing solutions** for something that’s important to their business or organization
* Some companies offer **prize money** &mdash; Netflix once offered **\$1,000,000** 
    * [Netflix wanted to get a 10% or better improvement in their model for determining whether people will like a movie, based on how they rated previous ones](https://netflixprize.com/rules.html)
    * Used to help make better recommendations to members
* Even if you do not win, **Kaggle** is a great way to get experience working on problems of current interest

<hr style="height:2px; border:none; color:#000; background-color:#000;">

# More Info 
* See Lesson 16 in [**Python Fundamentals LiveLessons** here on O'Reilly Online Learning](https://learning.oreilly.com/videos/python-fundamentals/9780135917411)
* See Chapter 16 in [**Python for Programmers** on O'Reilly Online Learning](https://learning.oreilly.com/library/view/python-for-programmers/9780135231364/)
* See Chapter 17 in [**Intro Python for Computer Science and Data Science** on O'Reilly Online Learning](https://learning.oreilly.com/library/view/intro-to-python/9780135404799/)
* Interested in a print book? Check out:

| Python for Programmers<br>(640-page professional book) | Intro to Python for Computer<br>Science and Data Science<br>(880-page college textbook)
| :------ | :------
| <a href="https://amzn.to/2VvdnxE"><img alt="Python for Programmers cover" src="../images/PyFPCover.png" width="150" border="1"/></a> | <a href="https://amzn.to/2LiDCmt"><img alt="Intro to Python for Computer Science and Data Science: Learning to Program with AI, Big Data and the Cloud" src="../images/IntroToPythonCover.png" width="159" border="1"></a>

>Please **do not** purchase both books&mdash;_Python for Programmers_ is a subset of _Intro to Python for Computer Science and Data Science_

&copy; 1992-2024 by Pearson Education, Inc. All Rights Reserved. The content in this notebook is based on the book [**Python for Programmers**](https://amzn.to/2VvdnxE).